# Config composition helper

In [45]:
from __future__ import annotations

from pathlib import Path

from hydra import compose, initialize_config_dir
from hydra.core.global_hydra import GlobalHydra


def locate_config_dir() -> Path:
    """Return the nearest `config` directory relative to the current working directory."""
    candidates = [
        Path.cwd() / "config",
        Path.cwd().parent / "config",
    ]
    for candidate in candidates:
        if candidate.is_dir():
            return candidate
    raise FileNotFoundError("Unable to locate a 'config' directory near the notebook.")


CONFIG_DIR = locate_config_dir()


def load_config(config_name: str = "config", overrides: list[str] | None = None):
    """Compose a Hydra config without leaving the global state initialized between runs."""
    overrides = overrides or []
    if GlobalHydra.instance().is_initialized():
        GlobalHydra.instance().clear()
    with initialize_config_dir(config_dir=str(CONFIG_DIR), job_name="notebook"):
        return compose(config_name=config_name, overrides=overrides)


print(f"Hydra configs will be loaded from: {CONFIG_DIR}")

Hydra configs will be loaded from: /home/ivan-yoga-ubuntu/VSCodeProjects/AMIE/config


In [46]:
# Example usage (uncomment and adjust config_name / overrides as needed):

cfg = load_config(config_name="backtest/default", overrides=[])
print(cfg)

{'backtest': {'initial_capital': 100000, 'slippage_bps': 5, 'fee_bps': 1, 'max_position_size': 1.0}}


/tmp/ipykernel_20526/1140843476.py:29: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(config_dir=str(CONFIG_DIR), job_name="notebook"):


In [47]:
from hydra import initialize, compose
from omegaconf import OmegaConf
from hydra.core.global_hydra import GlobalHydra

if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()

with initialize(version_base="1.3", config_path="../config"):
    cfg = compose(config_name="config")

print(OmegaConf.to_yaml(cfg))
assert cfg.run.seed == 42
print("✅ run.seed = 42")


data:
  n_ticks: 10000
  instruments:
  - BTC
  regimes:
  - type: calm
    duration: 2000
    volatility: 0.01
  - type: volatile
    duration: 2000
    volatility: 0.05
model:
  process_noise: 0.001
  observation_noise: 0.01
  warmup_period: 100
backtest:
  initial_capital: 100000
  slippage_bps: 5
  fee_bps: 1
  max_position_size: 1.0
run:
  seed: 42
  run_id: ???
paths:
  data_dir: data/
  results_dir: results/

✅ run.seed = 42


In [48]:
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf

if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()

with initialize(version_base="1.3", config_path="../config"):
    cfg = compose(config_name="config")

print("--- COMPOSED CFG ---")
print(OmegaConf.to_yaml(cfg))
print("Top-level keys:", list(cfg.keys()))


--- COMPOSED CFG ---
data:
  n_ticks: 10000
  instruments:
  - BTC
  regimes:
  - type: calm
    duration: 2000
    volatility: 0.01
  - type: volatile
    duration: 2000
    volatility: 0.05
model:
  process_noise: 0.001
  observation_noise: 0.01
  warmup_period: 100
backtest:
  initial_capital: 100000
  slippage_bps: 5
  fee_bps: 1
  max_position_size: 1.0
run:
  seed: 42
  run_id: ???
paths:
  data_dir: data/
  results_dir: results/

Top-level keys: ['data', 'model', 'backtest', 'run', 'paths']


In [4]:
from pathlib import Path
from hydra.core.global_hydra import GlobalHydra
from hydra import initialize_config_dir, compose
from omegaconf import OmegaConf

# 0) Point to the *absolute* config directory and verify the file exists
CONFIG_DIR = (Path.cwd() / "../config").resolve()
CONFIG_FILE = CONFIG_DIR / "config.yaml"
print("CWD:", Path.cwd())
print("CONFIG_DIR:", CONFIG_DIR)
print("config.yaml exists?", CONFIG_FILE.exists())
assert CONFIG_FILE.exists(), f"Missing: {CONFIG_FILE}"

# 1) Clear any previous Hydra singleton
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()

# 2) Initialize using ABSOLUTE directory (this API allows it)
with initialize_config_dir(version_base="1.3", config_dir=str(CONFIG_DIR)):
    cfg = compose(config_name="config")  # looks for CONFIG_DIR/config.yaml

# 3) Inspect
print(OmegaConf.to_yaml(cfg) or "<empty>")
# If your config.yaml has `defaults` and `_self_`, you should now see `run:` etc.
# Optional sanity check:
assert "run" in cfg and "seed" in cfg.run and cfg.run.seed == 42, f"Got: {OmegaConf.to_yaml(cfg)}"
print("✅ run.seed == 42")


CWD: /home/ivan-yoga-ubuntu/VSCodeProjects/AMIE/src
CONFIG_DIR: /home/ivan-yoga-ubuntu/VSCodeProjects/AMIE/config
config.yaml exists? True
data:
  instrument: BTC-USD
  regime_duration: 2000
  regimes:
  - volatility: 0.01
    spread: 25.0
    qty: 1.0
  - volatility: 0.05
    spread: 35.0
    qty: 1.2
  depth: 3
  level_spread: 10.0
  tick_interval_seconds: 1.0
  n_ticks: 10000
model:
  process_noise: 0.001
  observation_noise: 0.01
  warmup_period: 100
backtest:
  initial_capital: 100000
  slippage_bps: 5
  fee_bps: 1
  max_position_size: 1.0
run:
  seed: 42
  run_id: ???
paths:
  data_dir: data/
  results_dir: results/

✅ run.seed == 42


In [3]:
from amie.utils.logging import logger
logger.info("test", foo="bar")  # Should print structured JSON

from amie.utils.profiling import profile
@profile()
def slow_fn():
    import time; time.sleep(0.1)

slow_fn()  # Should log ~100ms

{"foo": "bar", "run_id": "eab3d233200849bd83b60abdd943c5ee", "timestamp": "2025-10-19T12:42:07.668226Z", "level": "info", "event": "test"}
{"function": "slow_fn", "duration_ms": 101.77513900089252, "run_id": "eab3d233200849bd83b60abdd943c5ee", "timestamp": "2025-10-19T12:42:07.771052Z", "level": "info", "event": "profiled_function"}


In [7]:
from amie.data.sources.synthetic_lob import SyntheticLOBGenerator

gen = SyntheticLOBGenerator(seed=42, config=cfg.data)
stream = list(gen.generate(int(cfg.data.n_ticks)))[:100]  # each item is (tick, lob)
assert len(stream) == 100

t0, lob0 = stream[0]
assert t0.price > 0
assert lob0.spread > 0

# Determinism
gen2 = SyntheticLOBGenerator(seed=42, config=cfg.data)
stream2 = list(gen2.generate(int(cfg.data.n_ticks)))[:100]

for (a_t, a_l), (b_t, b_l) in zip(stream, stream2):
    assert a_t.ts == b_t.ts
    assert a_t.price == b_t.price
    assert a_t.qty == b_t.qty
    assert a_t.side == b_t.side
    assert a_l.spread == b_l.spread
    assert a_l.bids[0] == b_l.bids[0]
    assert a_l.asks[0] == b_l.asks[0]

print("SyntheticLOBGenerator is deterministic ✅")


SyntheticLOBGenerator is deterministic ✅
